# Спеціальні розділи програмування
# Лабораторна робота №2
# Наука про дані: підготовчий етап


In [4]:
import pandas as pd
from urllib import request
from os import listdir, makedirs, walk, path
from shutil import rmtree
from datetime import datetime
from re import search

### Перевірка заданої папки на порожність. Якщо вона не порожня - папка очищується 

In [0]:
def check_folder(folder_path):
    if listdir(folder_path):
        rmtree(folder_path)
        makedirs(folder_path)

In [ ]:
check_folder('data')

### Скачування даних в папку data

In [5]:
def download_data(province_id):
    url = f'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={province_id}&year1=1981&year2=2024&type=Mean'

    try:
        vhi = request.urlopen(url).read().decode('utf-8')
        now = datetime.now()
        formatted_time = now.strftime('%d%m%Y%H%M%S')
        with open(f'data/vhi_id_{province_id}_{formatted_time}.csv', 'w') as f:
            f.write(vhi)
        
    except Exception as e:
        print(f'An error occurred while loading file with province id {province_id}')   

In [4]:
for i in range(1, 28):
    download_data(i)

### Конвертація файлів в датафрейми та препроцесинг

In [6]:
def create_df(file_path, norm_province_id):
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    temp_df = pd.read_csv(file_path, header = 1, names = headers)
    temp_df = temp_df.drop(temp_df.loc[temp_df['VHI'] == -1].index)
    temp_df.loc[0, 'Year'] = temp_df.loc[0, 'Year'][9:]
    temp_df = temp_df.drop(columns='empty').drop(temp_df.index[-1])
    temp_df['area'] = norm_province_id
    temp_df['Year'] = temp_df['Year'].astype(int)
    
    return temp_df

def get_paths(dir_path):
    file_paths = []
    for root, dirs, files in walk(dir_path):
        for i in files:
            file_paths.append(path.join(root, i))
            
    return file_paths  

def find_province_id(file_path):
    with open(file_path, 'r') as f:
        return int(search(r'Province\s*=\s*(\d+)', f.readline()).group(1))

In [7]:
replacements = {1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21, 11: 9, 12: 9,
              13: 10, 14: 11, 15: 12, 16: 13, 17: 14, 18: 15, 19: 16, 20: 25, 21: 17, 22: 18,
              23: 6, 24: 1, 25: 2, 26: 7, 27: 5}

adf = pd.concat([create_df(i, replacements[find_province_id(i)]) for i in get_paths('data')])

### Формування ряду VHI для області за вказаний рік

In [33]:
def form_vhi_series(df, province_id, year):
    filtered_df = df.loc[(df['area'] == province_id) & (df['Year'] == year)]
    final_s = pd.Series(filtered_df['VHI']).reset_index(drop=True)
    return final_s

In [34]:
form_vhi_series(adf, 2, 2020)

0     48.95
1     49.23
2     49.29
3     49.40
4     49.75
5     50.26
6     51.12
7     51.59
8     51.52
9     51.72
10    53.95
11    54.34
12    53.33
13    49.66
14    46.15
15    44.75
16    47.56
17    50.23
18    54.72
19    58.87
20    59.81
21    59.57
22    58.94
23    57.49
24    57.55
25    57.08
26    57.66
27    58.42
28    59.87
29    61.93
30    59.20
31    55.89
32    54.46
33    53.15
34    52.23
35    52.97
36    51.53
37    49.61
38    49.33
39    51.11
40    51.00
41    49.02
42    47.42
43    44.78
44    42.99
45    42.10
46    44.00
47    44.32
48    43.66
49    40.92
50    39.48
51    40.91
Name: VHI, dtype: float64

### Пошук екстремумів для вказаних областей та років, середнього, медіани

In [37]:
def find_mct(df, province_ids, year1, year2):
    temp_df = df.loc[(df['area'].isin(province_ids)) & (df['Year'].between(year1, year2))]
    temp_df = temp_df[['area', 'Year', 'VHI']]
    mct_vhi = df.groupby(['Year', 'area'])['VHI'].agg(['mean', 'min', 'max', 'median']).reset_index()
    mct_vhi.rename(columns={'mean': 'mean_VHI', 'min': 'min_VHI', 'max': 'max_VHI', 
                            'median': 'median_VHI'}, inplace=True)
    temp_df = temp_df.merge(mct_vhi, on=['Year', 'area'])
    temp_df = temp_df.drop(['VHI'], axis=1)
    temp_df = temp_df.drop_duplicates()
    temp_df.reset_index(drop=True, inplace=True)
    
    return temp_df 

In [38]:
find_mct(adf, [3, 8, 10], 1999, 2009)

,area,Year,mean_VHI,min_VHI,max_VHI,median_VHI
0,10,1999,47.952692,33.23,70.57,45.555
1,10,2000,42.436346,16.84,60.69,41.740
2,10,2001,53.611538,38.30,84.52,47.930
3,10,2002,49.776731,37.02,65.11,47.780
4,10,2003,47.911961,22.93,73.62,46.030
5,10,2004,64.158824,47.43,82.89,66.640
6,10,2005,49.579020,37.33,69.22,46.580
7,10,2006,46.344038,34.98,65.40,44.390
8,10,2007,41.782115,16.75,64.11,43.780
9,10,2008,50.656154,30.07,80.17,48.015


### Формування ряду VHI за вказаний діапазон років для вказаних областей

In [13]:
def form_vhi_df(df, province_ids, start_year, end_year):
    temp_df = df.loc[(df['area'].isin(province_ids)) & (df['Year'].between(start_year, end_year))]
    temp_df = temp_df[['area', 'Year', 'Week', 'VHI']]
    temp_df = temp_df.drop_duplicates()
    temp_df.reset_index(drop=True, inplace=True)
    
    return temp_df

In [14]:
form_vhi_df(adf, [4, 15, 21], 1991, 2008)

,area,Year,Week,VHI
0,21,1991,1.0,27.59
1,21,1991,2.0,31.77
2,21,1991,3.0,35.51
3,21,1991,4.0,39.04
4,21,1991,5.0,39.52
...,...,...,...,...
2686,4,2008,48.0,40.71
2687,4,2008,49.0,40.16
2688,4,2008,50.0,40.74
2689,4,2008,51.0,46.55


### Виявлення років, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей по Україні

In [50]:
def find_ed_years(df, percent):
    provinces_amount = int(27 * (percent/100))
    temp_df = df[['Year', 'area', 'VHI']]
    temp_df = temp_df[temp_df['VHI'] < 15]
    drought_count = temp_df.groupby('Year')['area'].nunique().reset_index()
    drought_count.columns = ['Year', 'Drought_Area_Count']
    ed_years = drought_count[drought_count['Drought_Area_Count'] > provinces_amount]
    temp_df = temp_df[temp_df['Year'].isin(ed_years['Year'])]
    temp_df.reset_index(drop=True, inplace=True)
    
    return temp_df

In [52]:
find_ed_years(adf, 10)

,Year,area,VHI
0,2000,9,12.51
1,2000,9,10.60
2,2000,9,11.20
3,2000,9,12.32
4,2000,9,14.65
...,...,...,...
83,2007,20,12.96
84,2007,20,13.48
85,2007,20,14.05
86,2007,20,14.41
